# Goal: Create the correlation and distance matrices for returns of PSE stocks over different windows

This cell is for importing necessary modules.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This cell is for reading the data file.

In [2]:
PSE_data = pd.read_csv(
    "C:\\Users\\Donna\\OneDrive - ASIAN INSTITUTE OF MANAGEMENT\\(A) PAPER\\AA Code\\PSE Stocks Data\\daily.csv"
)
PSE_data['date_id'] = pd.to_datetime(PSE_data['date_id'])
PSE_data = PSE_data.set_index("date_id")
df_filtered = PSE_data.drop(columns=[col for col in PSE_data.columns if not (col.startswith('PH_') and col.endswith('_P')) and col != 'date_id'])

This code is for removing the other rows that are not within the last 10 years of the data. I also used the date September 25, 2013 as the start date so that when I calculate log returns, I will have exactly 10 years worth of data. 

In [3]:
df_filtered = df_filtered.loc['2013-09-25':'2023-09-26']

## Cleaning data
This block is for cleaning the data and removing columns with too many null values.

In [4]:
def has_high_null_percentage(column, null_threshold):
    return column.isnull().mean() > null_threshold

# def has_consecutive_constant_values(column, threshold):
#     constant_streak = column.groupby((column != column.shift()).cumsum()).transform('size')
#     return constant_streak.max() > threshold

# def has_end_constant_values(column, end_threshold):
#     end_section = column.iloc[-int(len(column) * 0.05):]
#     return end_section.nunique() == 1

# def has_few_variations(column, variation_threshold):
#     return column.nunique() < variation_threshold

null_percentage_threshold = round(df_filtered.shape[0] * 0.85)
# consecutive_constant_threshold = int(len(df_filtered) * 0.35) 
# end_constant_threshold = 0.01
# variation_threshold = 50

columns_to_drop = []

for col in df_filtered.columns:
    if col == "date_id":  
        continue
    
    if (
        has_high_null_percentage(df_filtered[col], null_percentage_threshold) # or
        # has_consecutive_constant_values(df_filtered[col], consecutive_constant_threshold) or
        # has_end_constant_values(df_filtered[col], end_constant_threshold) or
        # has_few_variations(df_filtered[col], variation_threshold)
    ):
        columns_to_drop.append(col)

df_filtered = df_filtered.drop(columns=columns_to_drop)

In [5]:
df_filtered

,PH_PIP_P,PH_ASA_P,PH_ABS_P,PH_AGN_P,PH_APC_P,PH_CHP_P,PH_CEU_P,PH_CIR_P,PH_CAA_P,PH_EEQ_P,...,PH_MJC_P,PH_PCK_P,PH_MRP_P,PH_LOT_P,PH_BAG_P,PH_H2O_P,PH_PRC_P,PH_SSN_P,PH_SIN_P,PH_SHK_P
date_id,,,,,,,,,,,,,,,,,,,,,
2013-09-25,NaN,7.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,13.0,5.0,10.0,12.0,0.0,NaN
2013-09-26,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,12.0,6.0,10.0,11.0,0.0,NaN
2013-09-27,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,13.0,5.0,10.0,11.0,0.0,NaN
2013-09-30,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,12.0,5.0,10.0,11.0,0.0,NaN
2013-10-01,NaN,7.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,10.0,6.0,13.0,5.0,10.0,11.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,4.0,5.0,1.0,7.0,2.0,1.0,9.0
2023-09-21,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,4.0,5.0,1.0,7.0,2.0,1.0,9.0
2023-09-22,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,3.0,5.0,1.0,7.0,2.0,1.0,9.0


## Calculate returns from raw prices; log returns are probably easier, but you can try others like standardized returns if you have time

This cell is for calculating log returns.

In [6]:
# PSE_log_returns = df_filtered.copy()

# for col in PSE_log_returns.columns:
#     if col == "date_id":
#         continue  # Skip non-numeric columns
    
#     for row in range(1, len(PSE_log_returns)):  # Start at row 1 to avoid division by NaN
#         if pd.isna(PSE_log_returns.iloc[row, PSE_log_returns.columns.get_loc(col)]) or \
#            pd.isna(PSE_log_returns.iloc[row - 1, PSE_log_returns.columns.get_loc(col)]):
#             continue  # Skip computation if current or previous value is NaN
        
#         PSE_log_returns.iloc[row, PSE_log_returns.columns.get_loc(col)] = \
#             np.log(PSE_log_returns.iloc[row, PSE_log_returns.columns.get_loc(col)] /
#                    PSE_log_returns.iloc[row - 1, PSE_log_returns.columns.get_loc(col)])

PSE_log_returns = df_filtered.copy()
numeric_cols = PSE_log_returns.columns.difference(["date_id"])
PSE_log_returns[numeric_cols] = PSE_log_returns[numeric_cols].replace(0, np.nan)
PSE_log_returns[numeric_cols] = np.log(PSE_log_returns[numeric_cols] / PSE_log_returns[numeric_cols].shift(1))

I removed the the row for September 25, 2013, and I filled all the nan values with zero.

In [7]:
PSE_log_returns = PSE_log_returns.loc['2013-09-26':'2023-09-26']
PSE_log_returns

,PH_PIP_P,PH_ASA_P,PH_ABS_P,PH_AGN_P,PH_APC_P,PH_CHP_P,PH_CEU_P,PH_CIR_P,PH_CAA_P,PH_EEQ_P,...,PH_MJC_P,PH_PCK_P,PH_MRP_P,PH_LOT_P,PH_BAG_P,PH_H2O_P,PH_PRC_P,PH_SSN_P,PH_SIN_P,PH_SHK_P
date_id,,,,,,,,,,,,,,,,,,,,,
2013-09-26,NaN,-0.154151,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,-0.080043,0.182322,0.0,-0.087011,NaN,NaN
2013-09-27,NaN,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.080043,-0.182322,0.0,0.000000,NaN,NaN
2013-09-30,NaN,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,-0.080043,0.000000,0.0,0.000000,NaN,NaN
2013-10-01,NaN,0.154151,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.09531,0.000000,0.080043,0.000000,0.0,0.000000,NaN,NaN
2013-10-02,NaN,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20,0.0,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
2023-09-21,0.0,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
2023-09-22,0.0,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,-0.287682,0.000000,0.000000,0.0,0.000000,0.0,0.0


## Find an appropriate window size over which to calculate any given correlation matrix (literature suggests 3-6 months is typical, but double check). Construct the correlation matrix between returns for all valid stocks in each window. We have \~10 years of data (\~200 trading days per year), so this should make ~2000 windows.

This is for creating the correlation matrix. I used a window size of 6 months.

In [8]:
# correl_df = pd.DataFrame(columns=PSE_log_returns.columns).reset_index()

# for col in PSE_log_returns.columns:
#     correl_df[col] = PSE_log_returns[col].rolling(window=90).max()

# correl_df.reset_index()

In [9]:
correl_matrix = pd.DataFrame(index=PSE_log_returns.columns, columns=PSE_log_returns.columns)

for col1 in PSE_log_returns.columns:
    for col2 in PSE_log_returns.columns:
        if col1 != col2:
            # to_compare = PSE_log_returns[col2].rolling(window=180)
            rolling_corr = PSE_log_returns[col1].rolling(window=180).corr(PSE_log_returns[col2]) # cannot use corrcoef           
            correl_matrix.loc[col1, col2] = rolling_corr.iloc[-1]

In [10]:
correl_matrix

,PH_PIP_P,PH_ASA_P,PH_ABS_P,PH_AGN_P,PH_APC_P,PH_CHP_P,PH_CEU_P,PH_CIR_P,PH_CAA_P,PH_EEQ_P,...,PH_MJC_P,PH_PCK_P,PH_MRP_P,PH_LOT_P,PH_BAG_P,PH_H2O_P,PH_PRC_P,PH_SSN_P,PH_SIN_P,PH_SHK_P
PH_PIP_P,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.130492,-0.023933,...,0.0,0.0,NaN,0.0,-0.081123,NaN,0.055389,0.0,NaN,0.081986
PH_ASA_P,0.0,NaN,-0.146844,0.00346,NaN,NaN,0.067906,0.001276,-0.171624,-0.001871,...,0.0,0.002215,NaN,-0.001778,0.258256,NaN,-0.000346,0.002215,NaN,-0.092693
PH_ABS_P,0.0,-0.146844,NaN,0.046541,NaN,NaN,0.001168,-0.003463,-0.093871,0.005076,...,0.0,-0.006009,NaN,0.127213,-0.052425,NaN,0.068044,-0.006009,NaN,0.158659
PH_AGN_P,0.0,0.00346,0.046541,NaN,NaN,NaN,0.097924,-0.005029,0.004558,0.007372,...,0.0,-0.008727,NaN,0.007007,0.054649,NaN,0.14244,-0.008727,NaN,-0.067904
PH_APC_P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH_H2O_P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PH_PRC_P,0.055389,-0.000346,0.068044,0.14244,NaN,NaN,-0.00017,0.000503,-0.000502,0.057233,...,0.0,-0.184938,NaN,0.315127,-0.078262,NaN,NaN,0.000873,NaN,-0.032699
PH_SSN_P,0.0,0.002215,-0.006009,-0.008727,NaN,NaN,0.001086,-0.003219,-0.146282,0.004719,...,0.0,-0.005587,NaN,0.004486,-0.001278,NaN,0.000873,NaN,NaN,0.001292
PH_SIN_P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Display the resulting correlation matrix
correl_matrix = correl_matrix.astype(float) # .fillna(0)
correl_matrix.isnull().sum()

PH_PIP_P    147
PH_ASA_P    147
PH_ABS_P    147
PH_AGN_P    147
PH_APC_P    294
           ... 
PH_H2O_P    294
PH_PRC_P    147
PH_SSN_P    147
PH_SIN_P    294
PH_SHK_P    147
Length: 294, dtype: int64

In [12]:
# Display the resulting correlation matrix
correl_matrix = correl_matrix.astype(float).fillna(0)
correl_matrix.isnull().sum()

PH_PIP_P    0
PH_ASA_P    0
PH_ABS_P    0
PH_AGN_P    0
PH_APC_P    0
           ..
PH_H2O_P    0
PH_PRC_P    0
PH_SSN_P    0
PH_SIN_P    0
PH_SHK_P    0
Length: 294, dtype: int64

## Finding an appropriate window size and creating a covariance matrix

In [13]:
# correl_matrix = pd.DataFrame(index=PSE_log_returns.columns, columns=PSE_log_returns.columns)

# for col1 in PSE_log_returns.columns:
#     for col2 in PSE_log_returns.columns:
#         if col1 != col2:
#             # to_compare = PSE_log_returns[col2].rolling(window=180)
#             rolling_corr = PSE_log_returns[col1].rolling(window=180).corr(PSE_log_returns[col2]) # cannot use corrcoef           
#             correl_matrix.loc[col1, col2] = rolling_corr.iloc[-1]

The cell below is for creating the covariance matrix. I used a window size of 6 months.

In [ ]:
covar_matrix = pd.DataFrame(index=PSE_log_returns.columns, columns=PSE_log_returns.columns)

for col1 in PSE_log_returns.columns:
    for col2 in PSE_log_returns.columns:
        if col1 != col2:
            # to_compare = PSE_log_returns[col2].rolling(window=180)
            rolling_cov = PSE_log_returns[col1].rolling(window=180).cov(PSE_log_returns[col2]) # cannot use corrcoef           
            covar_matrix.loc[col1, col2] = rolling_cov.iloc[-1]

In [ ]:
covar_matrix

# Goal: Create the Minimum Spanning Tree

The code block below is for the imports for this portion of the code.

In [ ]:
import heapq

## Turn the covariance matrix into a distance matrix (see the Singapore papers, or some of the early Mantegna papers).

The blocks of code below turn the covariance matrix into a distance matrix.

In [ ]:
distance_matrix = 1 - np.round(1 - np.square(covar_matrix), 10)
distance_matrix = distance_matrix.to_numpy()

In [ ]:
distance_matrix

## Create the Minimum Spanning Tree. You may wish to revisit Prim’s or Kruskal’s algorithm from your data structures and algorithms class to calculate this.

I based my code from this reference:<br>
https://www.geeksforgeeks.org/prims-minimum-spanning-tree-mst-greedy-algo-5/<br>

I will be using Prim's algorithm.

In [ ]:
import numpy as np
import heapq

def prim_mst(distance_matrix):
    
    n = distance_matrix.shape[0]
    min_heap = []
    visited = [False] * n
    mst_edges = []
    visited[0] = True
    
    distance_matrix = distance_matrix.to_numpy()
    
    for i in range(1, n):
        heapq.heappush(min_heap, (distance_matrix[0, i], 0, i))
    
    while min_heap:
        weight, u, v = heapq.heappop(min_heap)
        
        if visited[v]:
            continue
        
        visited[v] = True
        mst_edges.append((u, v, weight))
        
        for i in range(n):
            if not visited[i]:
                heapq.heappush(min_heap, (distance_matrix[v, i], v, i))
    
    return mst_edges

distance_matrix = (distance_matrix + distance_matrix.T) / 2
mst_edges = prim_mst(distance_matrix)
print("Minimum Spanning Tree (MST) edges:")

for u, v, weight in mst_edges:
    print(f"Edge ({u}, {v}) with weight {weight:.6f}")